# --

In [1]:
import sys
from types import ModuleType, FunctionType
from gc import get_referents

# Custom objects know their class.
# Function objects seem to know way too much, including modules.
# Exclude modules as well.
BLACKLIST = type, ModuleType, FunctionType


def getsize(obj):
    """sum size of object & members."""
    if isinstance(obj, BLACKLIST):
        raise TypeError('getsize() does not take argument of type: '+ str(type(obj)))
    seen_ids = set()
    size = 0
    objects = [obj]
    while objects:
        need_referents = []
        for obj in objects:
            if not isinstance(obj, BLACKLIST) and id(obj) not in seen_ids:
                seen_ids.add(id(obj))
                size += sys.getsizeof(obj)
                need_referents.append(obj)
        objects = get_referents(*need_referents)
    return size

In [2]:
from ctypes import *
from ctypes.wintypes import *

FILE_ATTRIBUTE_REPARSE_POINT = 0x00400
INVALID_FILE_ATTRIBUTES = 0xFFFFFFFF

kernel32 = WinDLL('kernel32')
GetFileAttributesW = kernel32.GetFileAttributesW
GetFileAttributesW.restype = DWORD
GetFileAttributesW.argtypes = (LPCWSTR,) #lpFileName In

def islink(path):
    result = GetFileAttributesW(path)
    if result == INVALID_FILE_ATTRIBUTES:
        raise WinError()
    return bool(result & FILE_ATTRIBUTE_REPARSE_POINT)

islink(r'D:\Users\feelus\Local Settings')

True

In [3]:
import os
from stat import *

In [4]:
def scan(rootpath):
    root = {}
    #rootpath = 'D:\\'
    total_size = 0
    ts_printed = 0

    for curdir, indirs, infiles in os.walk(rootpath):
        path = curdir
        folders = []
        while path != rootpath:
            #print(path)
            path, folder = os.path.split(path)
            folders.append(folder)

        #print(curdir)
        #print(folders)
        curroot = root
        try:
            for i in reversed(folders):
                curroot = curroot[i]
        except:
            #print('path is apsent in tree:',curdir)
            continue

        for d in indirs:
            cur_d = os.path.join(curdir,d)
            try:
                mode = os.stat(cur_d).st_mode
                islnk = islink(cur_d)
            except:
                print("can't check stat of dir:",cur_d)
                continue
            if not S_ISDIR(mode) or islnk:
                print("this dir is not dir:",cur_d,mode)
                continue
            curroot[d]={}
        for f in infiles:
            cur_f = os.path.join(curdir,f)
            try:
                st = os.stat(cur_f)
                mode = st.st_mode
                size = st.st_size
                islnk = islink(cur_d)
            except:
                print("can't check stat of file:",cur_f)
                continue
            if not S_ISREG(mode):
                print("this file is not file:",cur_f,mode)
                continue
            if islnk:
                print("this file is not usual:",cur_f,mode)
            curroot[f]=[size]
            
            total_size+=size
            if ts_printed<int(total_size/1024/1024/1024):
                ts_printed = int(total_size/1024/1024/1024)
                print(ts_printed)
            
    return root


In [5]:
def tree_iterator(tree):
    for k,v in tree.items():
        if type(v)==dict:
            for path,v2 in tree_iterator(v):
                path.insert(0,k)
                yield path,v2
        else:
            yield [k],v

In [6]:
r = {'a':1,'b':{'c':2,'d':3}}
for x in tree_iterator(r):
    print(x)

(['a'], 1)
(['b', 'c'], 2)
(['b', 'd'], 3)


# --

In [14]:
rootpath = r'D:\Users\feelus\YandexDisk'
root = scan(rootpath)
len(root)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
can't check stat of file: D:\Users\feelus\YandexDisk\_programming\_lang_cpp\поиск файлов и msdn\_msdn_html\z_windows development\z_getting started\z_learn to program  for windows in c++\z_introduction to windows programming\Prepare Your Development Environment (Windows)_files\avatar.jpg
can't check stat of file: D:\Users\feelus\YandexDisk\_programming\_lang_cpp\поиск файлов и msdn\_msdn_html\z_windows development\z_getting started\z_learn to program  for windows in c++\z_introduction to windows programming\Prepare Your Development Environment (Windows)_files\Combined.css
can't check stat of file: D:\Users\feelus\YandexDisk\_programming\_lang_cpp\поиск файлов и msdn\_msdn_html\z_windows development\z_getting started\z_learn to program  for windows in c++\z_introduction to windo

76
77
78
can't check stat of file: D:\Users\feelus\YandexDisk\_programming_arxiv\_lang_cpp\стандарты и справочные руководства\Мюссер Д., Дердж Ж., Сейни А. C++ и STL. Справочное руководство (2-е издание, 2010)\Мюссер Д., Дердж Ж., Сейни А. C++ и STL. Справочное руководство (2-е издание, 2010).djvu
79
80
81
82
83
84
85
86
87
can't check stat of file: D:\Users\feelus\YandexDisk\Документы\programming\литература\интернет оффлайн\msdn_html\z_windows development\z_data acces and storage\z_Local File Systems\z_Directory Management\Directory Management Reference (Windows)_files\030c41d9079671d09a62d8e2c1db6973.gif
can't check stat of file: D:\Users\feelus\YandexDisk\Документы\programming\литература\интернет оффлайн\msdn_html\z_windows development\z_data acces and storage\z_Local File Systems\z_Directory Management\Directory Management Reference (Windows)_files\0d0542f5f45cf9fc7273abd11cf5c0ee.gif
can't check stat of file: D:\Users\feelus\YandexDisk\Документы\programming\литература\интернет офф

92
93
this dir is not dir: D:\Users\feelus\YandexDisk\программы-установщики\_бесплатные 16895
this dir is not dir: D:\Users\feelus\YandexDisk\программы-установщики\_бесплатные_portable 16895
this dir is not dir: D:\Users\feelus\YandexDisk\программы-установщики\_крякнутые_portable 16895
this file is not usual: D:\Users\feelus\YandexDisk\программы-установщики\[rutracker.org].t4229399 enwiki.torrent 33206
this file is not usual: D:\Users\feelus\YandexDisk\программы-установщики\_свободные.txt 33206
this file is not usual: D:\Users\feelus\YandexDisk\программы-установщики\не синхронизируемое.txt 33206
this file is not usual: D:\Users\feelus\YandexDisk\программы-установщики\проги.txt 33206
this file is not usual: D:\Users\feelus\YandexDisk\программы-установщики\.files\2017-07-08 23-59-14-dirs 33206
this file is not usual: D:\Users\feelus\YandexDisk\программы-установщики\.files\2017-07-08 23-59-14-files 33206
this file is not usual: D:\Users\feelus\YandexDisk\программы-установщики\.files\2017-

38

In [15]:
for k,v in root.items():
    if type(v)==dict:
        print(k+'/')
    else:
        print(k)

.files/
.sync/
crossingover/
Documents/
games/
mathematica-data/
multimedia/
notsynch_plan/
tmp/
_desktop/
_programming/
_programming_arxiv/
_target/
_математика/
_физика/
вещи, люди, места/
Документы/
Загрузки/
непонятное/
окюу3/
описание данных/
параплан/
программы-установщики/
Скриншоты/
универ/
Фотокамера/
худлит/
.lsdups-internal.exe.stackdump
desktop.ini
dictionary.c
kdevelop-alt-translations.patch
plan-2018-09.graphml
plan-2018-09.txt
qmake.qch
workdoc.txt
Корзина.yadiskTrash
сны.txt
созвездия.gif


In [16]:
getsize(root)/1024/1024

11.598617553710938

In [17]:
size_root={} # map размер -> множество файлов
for x,v in tree_iterator(root):
    size = v[0]
    file = tuple(x)
    #print(x,v,size)
    if size in size_root:
        size_root[size].add(file)
    else:
        size_root[size] = {file}

In [18]:
# map размер -> кол-во повторений
duplics = [(i,len(size_root[i])) for i in reversed(sorted(size_root.keys())) if len(size_root[i])!=1]
tot_s=0
for s,n in duplics:
    tot_s+=s*(n-1)
print(tot_s/1024/1024/1024,len(duplics))

2.249681477434933 10002


In [19]:
len(duplics)

10002

In [20]:
for s,n in duplics[0:100]:
    print(int(s/1024/1024))
    for p in sorted(size_root[s]):
        print('\t'+os.path.join(rootpath,*p))

57
	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\lib_QT\Qt_4.7_doc_ru\qt_ru.qch
	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\lib_QT\help\Qt_4.7_doc_ru1\qt_ru.qch
48
	D:\Users\feelus\YandexDisk\multimedia\статы - Видео Шведова\gl1\130916-133839.mpg
	D:\Users\feelus\YandexDisk\multimedia\статы - Видео Шведова\gl1\130923-112705.mpg
38
	D:\Users\feelus\YandexDisk\multimedia\статы - Видео Шведова\gl11\131118-093036.mpg
	D:\Users\feelus\YandexDisk\multimedia\статы - Видео Шведова\q4\68b.mpg
29
	D:\Users\feelus\YandexDisk\_математика\1 матан\[лекции Чирский] - ряды, фурье и дифуры.doc
	D:\Users\feelus\YandexDisk\multimedia\статы - Видео Шведова\gl1\130923-101517.mpg
20
	D:\Users\feelus\YandexDisk\_programming\_lang_cpp\lib_QT\[Макс Шлее] QT4.5. Профессиональное программирование на C++.djvu
	D:\Users\feelus\YandexDisk\_programming_arxiv\_разное\programming\QT\[Макс Шлее] QT4.5. Профессиональное программирование на C++.djvu
18
	D:\Users\feelus\YandexDisk\multimedia\статы - Видео Швед

In [29]:
recycle_duplics = []
other_duplics = []
for s,n in duplics:
    files = size_root[s]
    for k in files:
        if k[0]=='multimedia':
            recycle_duplics.append(s)
            break
    else:
        other_duplics.append(s)



In [66]:
for s,n in duplics:
    files = sorted(size_root[s])
    if not all([f[0]=='multimedia' for f in files]):
        continue
    if all([not(len(f)>=2 and f[1]=='vk-music') for f in files]):
        continue
    for f in files:
        if f[1]=='vk-music':
            print('#',os.path.join(rootpath,*f))
        else:
            os.remove(os.path.join(rootpath,*f))
            print('os.remove(r"'+os.path.join(rootpath,*f)+'")')
    print()
        


# D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\1516 Qazer - Viavana.mp3
# D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\и(2) Qazer\1749 Qazer - Viavana.mp3

os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Weekend Heroes - Black Ops! (Original Mix) (1).mp3")
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Weekend Heroes - Black Ops! (Original Mix).mp3")
# D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\клубные\609 Weekend Heroes - Black Ops! (Original Mix).mp3
# D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\клубные\611 Weekend Heroes - Black Ops! (Original Mix) (1).mp3

os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Kila - Dusty Wine Bottle (1).mp3")
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Kila - Dusty Wine Bottle.mp3")
# D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\641 Kila - Dusty Wine Bottle.mp3

os.remove(r"D:\Users\feelus\YandexDisk\multimedia\m-

os.remove(r"D:\Users\feelus\YandexDisk\multimedia\audio\vk2015\Василий К. & Интеллигенты - Шантарам.mp3")
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Василий К. & Интеллигенты - Шантарам.mp3")
# D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\7 рок средний\и(8) Василий К\1284 Василий К. & Интеллигенты - Шантарам.mp3

os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Титаник на Фонтанке - Наутилус Помпилиус-Воздух - Воздух.mp3")
# D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\7 рок средний\и(22) Бутусов Наутилус помпилиус\1657 Титаник на Фонтанке - Наутилус Помпилиус-Воздух - Воздух.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-muz\русские\Титаник на Фонтанке - Наутилус Помпилиус-Воздух - Воздух.mp3")

os.remove(r"D:\Users\feelus\YandexDisk\multimedia\m-пикник\Пикник - Недобитый романтик.mp3")
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Пикник - Недобитый романтик.mp3")
# D:\Users\feelus\YandexDisk\multimed

# D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\7 рок средний\и(72) Пикник\а(84) Танец волка\057 Пикник - Много дивного на свете .mp3

os.remove(r"D:\Users\feelus\YandexDisk\multimedia\m-пикник\Пикник - Телефон.mp3")
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Пикник - Телефон.mp3")
# D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\7 рок средний\и(72) Пикник\а(86) Иероглиф\399 Пикник - Телефон.mp3

os.remove(r"D:\Users\feelus\YandexDisk\multimedia\m-пикник\Пикник - Ночь (1).mp3")
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Пикник - Ночь (1).mp3")
# D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\7 рок средний\и(72) Пикник\а(82) дым\050 Пикник - Ночь.mp3

os.remove(r"D:\Users\feelus\YandexDisk\multimedia\m-пикник\2e298ea60cc1.mp3")
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Пикник - Очень интересно.mp3")
# D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\7 рок средний\и(72) Пикник\а(82) дым\047 Пик

os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Дергать - МГУ.mp3")
# D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\_про учёбу\170 Дергать - МГУ.mp3
# D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\_про учёбу\182 Дергать - МГУ (1).mp3

os.remove(r"D:\Users\feelus\YandexDisk\multimedia\audio\classic-rock\Track05.mp3")
# D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\и(8) Королевский филармонический оркестр\1465 Королевский филармонический оркестр - Без названия (1).mp3

os.remove(r"D:\Users\feelus\YandexDisk\multimedia\m-пикник\799a1fa93dca.mp3")
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Пикник - Новозеландская песня.mp3")
# D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\7 рок средний\и(72) Пикник\а(84) Танец волка\051 Пикник - Новозеландская песня.mp3

os.remove(r"D:\Users\feelus\YandexDisk\multimedia\audio\classic-rock\Track06.mp3")
# D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\и(8) Королевский ф

In [43]:
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Михаил Круг - Владимирский Централ (8x slow).mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-русское\440 Михаил Круг - Владимирский Централ (8x slow).mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Макс Фрай - «Чужак. Дебют в Ехо».mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\_рассказы\_аудиокниги\1514 Макс Фрай - «Чужак. Дебют в Ехо».mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Стивен Кинг - Стеклянный пол.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\_рассказы\_аудиокниги\1206 Стивен Кинг - Стеклянный пол.mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Аэроглиф - Никодим-Спутник.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\и(9) Аэроглиф\1299 Аэроглиф - Никодим-Спутник.mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Estas Tonne - Cosmic Fairytale.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\и(9) Estas Tonne\1583 Estas Tonne - Cosmic Fairytale.mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Doors - The End.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\1540 Doors - The End.mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Вивальди А. - The Four Seasons.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\и(39) Вивальди\концерты\899 Вивальди А. - The Four Seasons.mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Estas Tonne - The Winds that bring you Home.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\и(9) Estas Tonne\1584 Estas Tonne - The Winds that bring you Home.mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\шарль бодлер - парижский сплин (маленькие поэмы в прозе) .mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\_рассказы\_аудиокниги\394 шарль бодлер - парижский сплин (маленькие поэмы в прозе) .mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Genesis - Dancing With The Moonlit Knight.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\801 Genesis - Dancing With The Moonlit Knight.mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Most Heartful Themes Ever - Fear Not This Night (feat. Asja & Piano Version).mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\space\754 Most Heartful Themes Ever - Fear Not This Night (feat. Asja & Piano Version).mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Шостакович Дмитрий Дмитриевич - Фортепианное трио №2 e-moll, Op.67 - IV. Allegre.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\1235 Шостакович Дмитрий Дмитриевич - Фортепианное трио №2 e-moll, Op.67 - IV. Allegre.mp3
#os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Minitronix - Feelings.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\клубные\1217 Minitronix - Feelings.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Opeth - Hessian peel.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\и(5) Opeth\1347 Opeth - Hessian peel.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\разные ! инструментальные ! which house\Astrix Vs. Pixel - Here And There.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\клубные\622 Astrix Vs. Pixel - Here And There.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Public Service Broadcasting - Tomorrow.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\1324 Public Service Broadcasting - Tomorrow.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Tomcraft & Jerome Isma-Ae - Trouble In The Redlight District (Weekend Heroes Rem.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\клубные\613 Tomcraft & Jerome Isma-Ae - Trouble In The Redlight District (Weekend Heroes Rem.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Вавилонская библиотека (ТГ Варианты Возможны & ORT) - Х.Л. Борхес.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\_рассказы\_аудиокниги\781 Вавилонская библиотека (ТГ Варианты Возможны & ORT) - Х.Л. Борхес.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Ticon & Weekend Heroes - Teleport (Original Mix).mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\клубные\612 Ticon & Weekend Heroes - Teleport (Original Mix).mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Карлос Кастанеда-Учение Дона Хуана - Глава 2. Часть 1.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\_рассказы\и(2-9) Карлос Кастанеда\653 Карлос Кастанеда-Учение Дона Хуана - Глава 2. Часть 1.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\ә_Korpiklaani_ҽ - ә_Vesilahden verajilla_ҽ.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\и(88) Korpiklaani\a(2007) Tervaskanto\247 ә_Korpiklaani_ҽ - ә_Vesilahden verajilla_ҽ.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Black Sabbath - Heaven And Hell.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\рок\и(2) Black Sabbath\1077 Black Sabbath - Heaven And Hell.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Gilead - Herr Mannelig.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\и(8) Gilead\1511 Gilead - Herr Mannelig.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\НОЛЬ - На 90 градусов ниже нуля.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\7 рок средний\и(9) Ноль\867 НОЛЬ - На 90 градусов ниже нуля.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\разные ! инструментальные\Arvo Pärt - Festina Lente For Strings And Harp Ad Libitum.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\614 Arvo Pärt - Festina Lente For Strings And Harp Ad Libitum.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Симфоническое Кино - Спокойная ночь.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-русское\и(9) Симфоническое кино\1496 Симфоническое Кино - Спокойная ночь.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Mozart - Koncert fortepianowy C-dur- Allegro vivace assai.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\1059 Mozart - Koncert fortepianowy C-dur- Allegro vivace assai.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Korpiklaani - Sumussa hämärän aamun.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\и(88) Korpiklaani\a(2012) Manala\1342 Korpiklaani - Sumussa hämärän aamun.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\ә_Korpiklaani_ҽ - ә_Surma_ҽ.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\и(88) Korpiklaani\a(2011) Ukon wacka\292 ә_Korpiklaani_ҽ - ә_Surma_ҽ.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Поезд Родина - Белая Даль.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-русское\1159 Поезд Родина - Белая Даль.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Iron Maiden - When Two Worlds Collide.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\722 Iron Maiden - When Two Worlds Collide.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\ә_Korpiklaani_ҽ - ә_Kultanainen_ҽ.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\и(88) Korpiklaani\a(2009) Karkelo\239 ә_Korpiklaani_ҽ - ә_Kultanainen_ҽ.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Lacrimosa - Kaleidoskop.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\1436 Lacrimosa - Kaleidoskop.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Blue Stahli - Corner.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\983 Blue Stahli - Corner.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Lera Lynn - True Detective (Curly Tonic & Franz S. Bootleg).mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\и(7) Lera Lynn\1644 Lera Lynn - True Detective (Curly Tonic & Franz S. Bootleg).mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Stereophonics - In A Moment.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\986 Stereophonics - In A Moment.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Metalforce - Let The Battle Begin.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\рок тяжелый\1065 Metalforce - Let The Battle Begin.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\The Cracow Klezmer Band - Gil Gul (The Migration Of Souls).mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\164 The Cracow Klezmer Band - Gil Gul (The Migration Of Souls).mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Estas Tonne - The Song of the Golden Dragon.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\и(9) Estas Tonne\1582 Estas Tonne - The Song of the Golden Dragon.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Оргия Праведников - Вперед и вверх! (Для тех, кто видит сны).mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\7 рок средний\и(21) Калугин Оргия праведников\678 Оргия Праведников - Вперед и вверх! (Для тех, кто видит сны).mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Tung - Bullets.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\978 Tung - Bullets.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Fleur - Последний танец зимы.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\9 рок женский\и(9) Fleur\178 Fleur - Последний танец зимы.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\инструментальные ! Algambra\Algambra - The Turtle song.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\и(4) Algambra\1400 Algambra - The Turtle song.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Dj Маньяк и Рыбик - It's my Life.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\клубные\032 Dj Маньяк и Рыбик - It's my Life.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Estas Tonne - Cuban Rhapsody 1917 .mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\и(9) Estas Tonne\1587 Estas Tonne - Cuban Rhapsody 1917.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Fleur - Баллада о белых крыльях и алых лепестках.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\9 рок женский\и(9) Fleur\1638 Fleur - Баллада о белых крыльях и алых лепестках.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Ludovico Einaudi - Night.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\и(6) Ludovico Einaudi\1594 Ludovico Einaudi - Night.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Jay-Jay Johanson - Drowsy _ Too Young To Say Good Night.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\1309 Jay-Jay Johanson - Drowsy _ Too Young To Say Good Night.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Yoko Kanno, Origa - Rise.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\рок женский\1213 Yoko Kanno, Origa - Rise.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Вальс в Конго - Пальто (live).mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\и(2) Вальс в Конго\1445 Вальс в Конго - Пальто (live).mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\ә_Korpiklaani_ҽ - ә_Hengettömiltä hengiltä_ҽ.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\и(88) Korpiklaani\a(2003) Spirit of the forest\238 ә_Korpiklaani_ҽ - ә_Hengettömiltä hengiltä_ҽ.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Ария - Осколок льда.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\русское\8 рок тяжелый\и(9) Кипелов - Ария\806 Ария - Осколок льда.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Caprice - Sage.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\инструментал-зарубежное\и(3) Caprice\1204 Caprice - Sage.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Zodiac - Зодиак.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\мелодии\1321 Zodiac - Зодиак.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Modana ft. Carlprit - Party Crash (Extended Mix) .mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\клубные\606 Modana ft. Carlprit - Party Crash (Extended Mix) .mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\Wavefall - Tuty F. Fruity.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\зарубежное\рок тяжелый\1132 Wavefall - Tuty F. Fruity.mp3
os.remove(r"D:\Users\feelus\YandexDisk\multimedia\vk-misic-2018\F-777 - Space Battle.mp3") #D:\Users\feelus\YandexDisk\multimedia\vk-music\интрументал\и(2) F-777\1478 F-777 - Space Battle.mp3

In [ ]:
import gc
gc.collect()